Projeto Final Pós Graduação Big Data - Senac Rio - 2020.

Processo de importação das Bibliotecas utilizadas no durante todo o projeto.

In [1]:
#Importação das bibliotecas utilizadas para manipulação dos dados das paginas Web.

from bs4 import BeautifulSoup
import urllib.request
import shutil
import os
from datetime import datetime
import pandas as pd
import pyspark
import zipfile
import sqlalchemy
import pymysql
import json
import numpy as np
import matplotlib.pyplot as plt

Indicação do diretório para download dos arquivos Inep-Enem. 

In [ ]:
#Indicação do diretório desejado para download dos arquivos. 
#No exemplo, estamos salvando no diretório: C:\Users\Ricardo\Downloads

diretorio = input('Digite o diretório desejado para salvar os arquivos: ')
os.chdir(r'{}'.format(diretorio))
os.getcwd( )

In [ ]:
#Apos indicar o diretorio desejado, como boa pratica, sugerimos a criação da pasta onde será salvos os arquivos.
#No exemplo, estamos salvando na pasta: MicrodadosEnem.'''

url = 'http://inep.gov.br/microdados'

pasta = input('Após indicação do diretório {}, informe o nome da pasta desejada para salvar os arquivos baixados da pagina {}: '.format(diretorio, url))

try:
    os.mkdir(pasta)
    print('Pasta {} criada com sucesso!'.format(pasta))
except FileExistsError as e:
    print(f'Pasta {pasta} já existe')

In [ ]:
#Indicação do diretório onde será salvo os arquivos baixados da url.

os.chdir(r'{}\{}'.format(diretorio, pasta))
print('Os arquivos serão salvos no seguinte diretorio:', format(os.getcwd()))

Processo de Web Scraping no site do Inep, buscando os microdados do Enem.

In [ ]:
#Criação da variável que receberá todos os dados correspondente a varíavel "url".

page = urllib.request.urlopen(url)

In [ ]:
#Criação da variável "soup", do pacote BeatifulSoup, que apresenta a estutura HTML da pagina que será trabalhada.

soup = BeautifulSoup(page,'html.parser')
print(soup)

In [ ]:
#No loop "for" abaixo, o realizaremos o filtro das tags desejadas.

for link in soup.findAll('a'):
    theLink = link.get('href')
    name = link.string
    
    print(theLink)
    print(name)

In [ ]:
#No processo abaixo, realizaremos o filtro das linhas, cujo o o tipo do arquivo desejado é *.zip.

nomeArquivo = []
typeFile = '.zip'
i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            print(theLink)
            print(type(theLink))
            print(name)
            print(type(name))
            nomeArquivo.append(theLink[39:-4].replace('/','_'))
            print(type(nomeArquivo))

In [ ]:
#Processo de download dos arquivos desejados e salvos na na pasta indicada.

i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            if name is None:
                title = 'SemTítulo' + typeFile
            else:
                title = nomeArquivo[i] + typeFile
                i +=1
            try:
                with open(os.getcwd() + '/' + title,'r') as fileExists:
                    print('Arquivo ' + title + ' existente no diretório '+ os.getcwd() + ' - ' + str(datetime.now()))
                    fileExists.close()
            except FileNotFoundError:
                print('Download Iniciado - {}: '.format(datetime.now()) + title)
                docFile = open(os.getcwd() + '/' + title,'wb')
                docFile.write(urllib.request.urlopen(theLink).read())
                docFile.close()
                print('Download Concluído em {}'.format(datetime.now()))
print('Não há mais arquivos para download')

In [ ]:
#Apresentação do tamanho dos arquivos baixados.

print('Apresentação Arquivos Enem baixados no díretório: ' + os.getcwd())
contador = 0
tamanhoTotal = 0
tipo = '.zip'

def formatarTamanho(tamanho):
    base = 1024
    kilo = base
    mega = base ** 2
    giga = base ** 3
    tera = base ** 4
    
    if tamanho < kilo:
        tamanho = base
        texto = 'B'
    elif tamanho < mega:
        tamanho /= kilo
        texto = 'K'
    elif tamanho < giga:
        tamanho /= mega
        texto = 'M'
    else:
        tamanho < tera
        tamanho /= giga
        texto = 'G'
    tamanho = round(tamanho, 2)
    return f'{tamanho} {texto}'

for raiz, diretorios, arquivos in os.walk(os.getcwd()):
    for arquivo in arquivos:
        if tipo in arquivo:
            contador +=1 
            caminho = os.path.join(raiz, arquivo)
            nome, ext = os.path.splitext(arquivo)
            tamanho = os.path.getsize(caminho)
            tamanhoTotal += tamanho
                                
            if tamanho > 0:
                print()
                print('Arquivo encontrado: ', arquivo)
                print('Caminho: ', caminho)
                print('Nome: ', nome)
                print('Extensão: ', ext)
                print('Tamanho: ', formatarTamanho(tamanho))

print()
print(f'{contador} arquivo(s) encontrado(s).' + 'Tamanho Total: ' + formatarTamanho(tamanhoTotal))

Processo de Conexão ao Mysql e criação do banco de dados que receberá os microdados dos anos de 2015 a 2019.

In [ ]:
# Criando um database.

conexao = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'admin')

cursor = conexao.cursor()

try:
    cursor.execute('CREATE DATABASE dbenem')
    print('Banco de Dados dbenem criado com sucesso!')
except:
    print('Banco de Dados dbenem já existe!')

In [ ]:
# Conectando ao Banco de Dados enemdb.

cursor.execute("SHOW DATABASES")
print('Databases MySql:')
for x in cursor:
  print(x)


cursor.connection.select_db('dbenem')
print('')
print('Conexão com o banco de dados dbenem')

In [2]:
from pyspark import SparkContext
sc = SparkContext()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
# No Processo abaixo, importaremos para o dataframe, apenas para conhecer os dados e verificar o separador do microdado.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
enem2015 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=",", inferschema='true', encoding = 'ISO-8859-1', ).load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2015.csv')
print(f'Dataframe enem2015 criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2016 iniciada- {datetime.now()}.')
enem2016 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\microdados_enem_2016.csv')
print(f'Dataframe enem2016 criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
enem2017 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2017.csv')
print(f'Dataframe enem2017 criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
enem2018 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2018.csv')
print(f'Dataframe enem2018 criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
enem2019 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2019.csv')
print(f'Dataframe enem2019 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Dataframes criados com sucesso - {datetime.now()}.')

Criação Dataframes Iniciado - 2020-08-14 18:18:43.229565.

Criação Dataframe do Enem 2015 iniciada - 2020-08-14 18:18:43.231544.
Dataframe enem2015 criado com sucesso - 2020-08-14 18:19:32.748865.

Criação Dataframe do Enem 2016 iniciada- 2020-08-14 18:19:32.749860.
Dataframe enem2016 criado com sucesso - 2020-08-14 18:20:29.942338.

Criação Dataframe do Enem 2017 iniciada - 2020-08-14 18:20:29.943333.
Dataframe enem2017 criado com sucesso - 2020-08-14 18:21:05.075237.

Criação Dataframe do Enem 2018 iniciada - 2020-08-14 18:21:05.076234.
Dataframe enem2018 criado com sucesso - 2020-08-14 18:21:35.011080.

Criação Dataframe do Enem 2019 iniciada - 2020-08-14 18:21:35.012076.
Dataframe enem2019 criado com sucesso - 2020-08-14 18:22:01.726288.


Dataframes criados com sucesso - 2020-08-14 18:22:01.726288.


In [5]:
#Importando dos Dataframes para tabelas no mysql

print(f'Iniciando a criação e importação dos dados do Microdados Enem para tabela mysql - {datetime.now()}.')
print('')

print('')
print(f'Criação da Tabela tb_enem2015 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2015 para a Tabela tb_enem2015 iniciada - {datetime.now()}.')
enem2015.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2015',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2015 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2016 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2016 para a Tabela tb_enem2016 iniciada - {datetime.now()}.')
enem2016.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2016',
    user='root', password='admin').mode('overwrite').save()  
print(f'Importação para a Tabela tb_enem2016 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2017 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2017 para a Tabela tb_enem2017 iniciada - {datetime.now()}.')
enem2017.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2017',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2017 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2018 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2018 para a Tabela tb_enem2018 iniciada - {datetime.now()}.')
enem2018.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2018',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2018 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2019 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2019 para a Tabela tb_enem2019 iniciada - {datetime.now()}.')
enem2019.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2019',
    user='root', password='admin').mode('overwrite').save()    
print(f'Importação para a Tabela tb_enem2019 com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Tabelas e Importações realizados com sucesso - {datetime.now()}.')

Iniciando a criação e importação dos dados do Microdados Enem para tabela mysql - 2020-08-14 18:24:30.703137.


Criação da Tabela tb_enem2015 iniciada - 2020-08-14 18:24:30.703137.
Importação do Dataframe enem2015 para a Tabela tb_enem2015 iniciada - 2020-08-14 18:24:30.703137.
Importação para a Tabela tb_enem2015 com sucesso - 2020-08-14 18:54:14.514733.

Criação da Tabela tb_enem2016 iniciada - 2020-08-14 18:54:14.516727.
Importação do Dataframe enem2016 para a Tabela tb_enem2016 iniciada - 2020-08-14 18:54:14.516727.
Importação para a Tabela tb_enem2016 com sucesso - 2020-08-14 19:30:42.159196.

Criação da Tabela tb_enem2017 iniciada - 2020-08-14 19:30:42.160617.
Importação do Dataframe enem2017 para a Tabela tb_enem2017 iniciada - 2020-08-14 19:30:42.161615.
Importação para a Tabela tb_enem2017 com sucesso - 2020-08-14 19:53:11.265111.

Criação da Tabela tb_enem2018 iniciada - 2020-08-14 19:53:11.266107.
Importação do Dataframe enem2018 para a Tabela tb_enem2018 iniciada - 2020-08-